사전순 정렬 대신 natural sort를 사용한다.

In [159]:
# cell 1

import re

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]


먼저 파일을 읽어서 카테고리 이름을 요소로 갖는 string list를 만든다.

In [160]:
# cell 2

import os

path = '/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data'
folder_list = os.listdir(path)

category = []
# 카테고리 string 저장할 list

for i in folder_list:
    if not i.endswith('.DS_Store'):
    	# .DS_Store 빼고 추가
        category.append(i)
        
category.sort(key=natural_keys)

print(category)

['child', 'culture', 'economy', 'education', 'health', 'life', 'person', 'policy', 'society']


각 카테고리 안에 있는 파일 이름을 요소로 갖는 2D string list를 만든다.

In [161]:
# cell 3

fileName2D = []
# row: categoryIdx, column: 해당 카테고리의 파일 이름

for categoryIdx in range(len(category)):
    path = '/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data/'+category[categoryIdx]
    file_list = os.listdir(path)
    #해당 카테고리 안에 있는 파일명이 담긴 string list

    txt_list = []
    for i in file_list:
        if i.endswith('.txt') and not i.endswith('.DS_Store'):
            #파일 형식이 .txt로 끝나는 파일 이름만 배열에 추가
            txt_list.append(i)

    txt_list.sort(key=natural_keys)
    # natural sort
    fileName2D.append(txt_list)

print("카테고리마다의 파일 개수")
for i in fileName2D:
    print(len(i), end=" ")

# print("\n",fileName2D)


카테고리마다의 파일 개수
128 219 166 120 191 112 172 252 327 

### 모든 파일의 형태소를 요소로 갖는 중복없는 corpus 생성

row: docNum, 0번 column: category,  나머지 column: corpus 인 2D Dataframe을 만든다

빈 DataFrame을 생성했을 때 행과 열이 둘 다 없으면 데이터를 추가하거나 수정할 수 없다.

열 정보를 갖고 있어야 행 데이터를 추가할 수 있기 때문에 첫 column인 category를 추가해준다.

데이터 프레임 값 변경: df.loc[2, 'A'] = 3000

if morp == '':일때 컨티뉴 해주어야하는데 시간상  break로 첫 문단만 가져옴

In [162]:
# cell 4

import pandas as pd
import numpy as np

TT = pd.DataFrame(columns=range(1))
TT.columns = ['category']
# term table

docNum = 0
#문서 번호

for categoryIdx in range(len(category)):
    for fileIdx in range(len(fileName2D[categoryIdx])):
        file = open("/Users/jeongdeok/Downloads/2022_Fall_Student_Data/8/Corpus/Input_Data/"+category[categoryIdx]+"/"+fileName2D[categoryIdx][fileIdx], "r")

        lines = file.readlines()
        #한 줄씩 string으로 list에 저장

        docRow = "Doc"+str(docNum)
        #row 이름 ex) Doc3

        TT.loc[docRow] = [0 for i in range(len(TT.columns))]
        # ex)Doc3 = [0, 0, 0, 0](열 개수만큼 0을 만들어 list로 반환)

        TT.loc[docRow, 'category'] = categoryIdx
        # 해당 문서의 카테고리 값에 categoryIdx 저장
        
        for line in lines:
            morp = line.split('\t')[-1].split('+')[0].strip('\n')
            #print(morp)
            # '\t'를 기준으로 뒤, '+'를 기준으로 앞의 string을 잘라 list에 저장
            if morp == '':
                break
########################################## continue로 바꿔야 함 ########################################
            # 빈 string이면 무시

            if morp in TT.columns:
                TT.loc[docRow,morp] += 1
            else:
                TT[morp] = 0
                TT.loc[docRow,morp] += 1
            # TF columns에서 중복 확인, 중복 있으면 값만 1 증가시킴, 없으면 열 추가 후 값 1 대입 

        docNum += 1
                
TT

,category,與/SL,성폭행/NNG,`/SW,거세/NNG,전면확대/NNG,추진/NNG,‘/SN,지원’/NNG,위하/VV,...,싸움/NNG,승소/NNG,스웨덴/NNP,43/SN,달러/NNB,국고보조금/NNG,농아인협회/NNG,광주인화학교/NNP,행정실장/NNG,감형/NNG
Doc0,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1682,8,0,0,0,0,0,0,0,0,0,...,1,1,0,0,0,0,0,0,0,0
Doc1683,8,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
Doc1684,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1685,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0


### TF값 저장

(category column 에는 0 저장)

In [163]:
from collections import Counter
# TT.loc['tf'] = TT.drop('category', axis=1).sum(axis=0)

TT.loc['tf'] = TT.sum(axis=0)
TT.loc['tf','category'] = 0
TT


,category,與/SL,성폭행/NNG,`/SW,거세/NNG,전면확대/NNG,추진/NNG,‘/SN,지원’/NNG,위하/VV,...,싸움/NNG,승소/NNG,스웨덴/NNP,43/SN,달러/NNB,국고보조금/NNG,농아인협회/NNG,광주인화학교/NNP,행정실장/NNG,감형/NNG
Doc0,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1683,8,0,0,0,0,0,0,0,0,0,...,0,0,1,1,1,0,0,0,0,0
Doc1684,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1685,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
Doc1686,8,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1


### DF값 저장
TT를 .copy()로 깊은 복사한 CC 사용

In [171]:
# TT.loc['df'] = [0 for i in range(len(TT.columns))]
CC = TT.copy()
colIdx = TT.shape[1]
# TT dataframe column 개수
tf_remove = CC.drop('tf')
df_list = []
# 새로 추가할 row인 'df'에 저장할 list

for c in range(colIdx):
    df_list.append(len(tf_remove.loc[tf_remove.iloc[:,c] != 0]))
    # 모든 column을 다 돌면서 값이 0이 아닌 row가 몇개 있는지 df_list에 저장

CC.loc['df'] = df_list
CC.loc['df','category'] = 0
# TT.loc['df'].iloc[1] = float(2222)
# TT
CC

,category,與/SL,성폭행/NNG,`/SW,거세/NNG,전면확대/NNG,추진/NNG,‘/SN,지원’/NNG,위하/VV,...,싸움/NNG,승소/NNG,스웨덴/NNP,43/SN,달러/NNB,국고보조금/NNG,농아인협회/NNG,광주인화학교/NNP,행정실장/NNG,감형/NNG
Doc0,0,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1,0,0,0,0,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
Doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1684,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1685,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
Doc1686,8,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,1
tf,0,4,25,11,3,2,23,186,2,39,...,1,1,1,1,1,1,1,1,1,1


### 카테고리 label을 갖고 있는 (n,1) dataframe 생성

In [165]:
label = pd.DataFrame(CC['category'])
label

,category
Doc0,0
Doc1,0
Doc2,0
Doc3,0
Doc4,0
...,...
Doc1684,8
Doc1685,8
Doc1686,8
tf,0


### TF 값 기준으로 column 정렬
내림차순으로 정렬 후 1000번째 이후 요소는 삭제한다.

In [170]:
CC_sorted = CC.drop('category', axis=1).sort_values(by='tf', axis= 1, ascending=False)
CC_sorted = CC_sorted.iloc[:,:1000]
# .loc은 row, column 이름으로, 
# .iloc은 row, column 인덱스로. 
CC_sorted

,'/SS,‘/SL,‘/SN,장애인/NNG,"""/SS",“/SL,지원/NNG,위하/VV,내년/NNG,서울시/NNP,...,접근성/NNG,못하/VV,원전/NNG,죽음/NNG,농아인/NNG,보호견/NNG,걱정/NNG,기부/NNG,최대행사/NNG,한인/NNG
Doc0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Doc1684,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1685,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Doc1686,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
tf,292,212,186,185,132,57,40,39,38,38,...,2,2,2,2,2,2,2,2,2,2


### ITF 함수 정의
